# NV-Ingest: Python Client Quick Start Guide

This notebook provides a quick start guide to using the NV-Ingest Python API to create a client that interacts with a running NV-Ingest cluster. It will walk through the following:

- Instantiate a client object
- Define the task configuration for an NV-Ingest job
- Submit a job the the NV-Ingest cluster
- Retrieve completed results
- Investigate the multimodal extractions


Specify a few parameters to connect to our nv-ingest cluster and a notional document to guide the examples.

In [ ]:
import os

# client config
HTTP_HOST = os.environ.get('HTTP_HOST', "localhost")
HTTP_PORT = os.environ.get('HTTP_PORT', "7670")
TASK_QUEUE = os.environ.get('TASK_QUEUE', "morpheus_task_queue")

# minio config
MINIO_ACCESS_KEY = os.environ.get('MINIO_ACCESS_KEY', "minioadmin")
MINIO_SECRET_KEY = os.environ.get('MINIO_SECRET_KEY', "minioadmin")

# time to wait for job to complete
DEFAULT_JOB_TIMEOUT = 90

# sample input file and output directory
SAMPLE_PDF = "/workspace/nv-ingest/data/multimodal_test.pdf"

## The NV-Ingest Python Client

In [ ]:
from base64 import b64decode
import time

from nv_ingest_client.client import NvIngestClient
from nv_ingest_client.message_clients.rest.rest_client import RestClient
from nv_ingest_client.primitives import JobSpec
from nv_ingest_client.primitives.tasks import DedupTask
from nv_ingest_client.primitives.tasks import EmbedTask
from nv_ingest_client.primitives.tasks import ExtractTask
from nv_ingest_client.primitives.tasks import FilterTask
from nv_ingest_client.primitives.tasks import SplitTask
from nv_ingest_client.primitives.tasks import StoreTask, StoreEmbedTask
from nv_ingest_client.primitives.tasks import VdbUploadTask
from nv_ingest_client.util.file_processing.extract import extract_file_content

from IPython import display

Load a sample PDF to demonstrate NV-Ingest usage.

In [ ]:
file_content, file_type = extract_file_content(SAMPLE_PDF)

Initialize a client that will submit jobs to our NV-Ingest cluster.

In [ ]:
client = NvIngestClient(
    message_client_allocator=RestClient,
    message_client_hostname=HTTP_HOST,
    message_client_port=HTTP_PORT,
    message_client_kwargs=None,
    msg_counter_id="nv-ingest-message-id",
    worker_pool_size=1,
)

A `JobSpec` is a specification for creating a job for submission to the NV-Ingest microservice. It accepts the following parameters:

- `document_type` : The file type of the file to be ingested.
- `payload` : A base64 encoded string of the file to be ingested.
- `source_id` : An identifier that maps to the file, our example uses the filename here.
- `source_name` : The name of the source for this ingest job, again, we use the filename.
- `extented_options` : A dictionary of additional options to pass in to the ingest job, we pass in informations allowing us to conduct performance tracing of the job.


In [ ]:
job_spec = JobSpec(
    document_type=file_type,
    payload=file_content,
    source_id=SAMPLE_PDF,
    source_name=SAMPLE_PDF,
    extended_options={
        "tracing_options": {
            "trace": True,
            "ts_send": time.time_ns(),
        }
    },
)

Each ingest job will include a set of tasks. These tasks will define the operations that will be performed during ingestion. This allows each job to potentially have different ingestion instructions. Here we define a simple extract oriented job, but the full list of supported options are contained below:

- `ExtractTask` : Performs multimodal extractions from a document, including text, images, and tables.
- `TableExtractTask`: Extracts the content from tables.
- `ChartExtractTask`: Extracts the content from charts.
- `SplitTask` : Chunk the text into smaller chunks, useful for storing in a vector database for retrieval applications.
- `Dedup` : Identifies duplicate images in document that can be filtered to remove data redundancy.
- `Filter` : Filters out images that are likely not useful using some heuristics, including size and aspect ratio.
- `EmbedTask` : Pass the text or table extractions through `"nvidia/nv-embedqa-e5-v5` NIM to obtain its embeddings.
- `Store` : Save the extracted tables or images to an S3 compliant object store like MinIO.
- `Upload` : Save embeddings, chunks, and metadata to a Milvus vector database.

After defining the ingestion tasks, they must be added to the job specification.

In [ ]:
extract_task = ExtractTask(
    document_type=file_type,
    extract_text=True,
    extract_images=True,
    extract_tables=True,
    text_depth="document",
    extract_tables_method="yolox",
)

dedup_task = DedupTask(
    content_type="image",
    filter=True,
)

job_spec.add_task(extract_task)
job_spec.add_task(dedup_task)

A job identifier is created for the job specification. This is used to retrieve the results upon completion.

With the `job_id`, the job is submitted to the NV-Ingest microservice. When the job is complete, the results are fetched.

Note, many jobs can be submitted and asynchronous execution is supported.

In [ ]:
job_id = client.add_job(job_spec)
client.submit_job(job_id, TASK_QUEUE)
generated_metadata = client.fetch_job_result(
    job_id, timeout=DEFAULT_JOB_TIMEOUT
)[0]

## Explore the Outputs

Let's explore elements of the NV-Ingest output. When data flows through an NV-Ingest pipeline, a number of extractions and transformations are performed. As the data is enriched, it is stored in rich metadata hierarchy. In the end, there will be a list of dictionaries, each of which represents a extracted type of information. The most common elements to extract from a dictionary in this hierarchy are the extracted content and the text representation of this content. The next few cells will demonstrate interacting with the metadata, pulling out these elements, and visualizing them. Note, when there is a `-1` value present, this represents non-applicable positional resolution. Positive numbers represent valid positional data.

For a more complete description of metadata elements, view the data dictionary.

[https://github.com/NVIDIA/nv-ingest/blob/main/docs/content-metadata.md](https://github.com/NVIDIA/nv-ingest/blob/main/docs/content-metadata.md)

In [ ]:
def redact_metadata_helper(metadata: dict) -> dict:
    """A simple helper function to redact `metadata["content"]` so improve readability."""
    
    text_metadata_redact = text_metadata.copy()
    text_metadata_redact["content"] = "<---Redacted for readability--->"
    
    return text_metadata_redact

### Explore Output - Text

This cell depicts the full metadata hierarchy for a text extraction with redacted content to ease readability. Notice the following sections are populated with information:

- `content` - The raw extracted content, text in this case - this section will always be populated with a successful job.
- `content_metadata` - Describes the type of extraction and its position in the broader document - this section will always be populated with a successful job.
- `source_metadata` - Describes the source document that is the basis of the ingest job.
- `text_metadata` - Contain information about the text extraction, including detected language, among others - this section will only exist when `metadata['content_metadata']['type'] == 'text'`

In [ ]:
text_metadata = generated_metadata[3]["metadata"]
redact_metadata_helper(text_metadata)

View the text extracted from the sample document.

In [ ]:
text_metadata["content"]

### Explore Output - Tables

This cell depicts the full metadata hierarchy for a table extraction with redacted content to ease readability. Notice the following sections are populated with information:

- `content` - The raw extracted content, a base64 encoded image of the extracted table in this case - this section will always be populated with a successful job.
- `content_metadata` - Describes the type of extraction and its position in the broader document - this section will always be populated with a successful job.
- `source_metadata` - Describes the source and storage path of an extracted table in an S3 compliant object store.
- `table_metadata` - Contains the text representation of the table, positional data, and other useful elements - this section will only exist when `metadata['content_metadata']['type'] == 'structured'`.

Note, `table_metadata` will store chart and table extractions. The are distringuished by `metadata['content_metadata']['subtype']`

In [ ]:
table_metadata = generated_metadata[4]["metadata"]
redact_metadata_helper(table_metadata)

Visualize the table contained within the extracted metadata.

In [ ]:
display.Image(b64decode(table_metadata["content"]))

View the corresponding text that maps to this table. This text could be embedded to support multimodal retrieval workflows.

In [ ]:
table_metadata["table_metadata"]["table_content"]

### Explore Output - Charts

This cell depicts the full metadata hierarchy for a chart extraction with redacted content to ease readability. Notice the following sections are populated with information:

- `content` - The raw extracted content, a base64 encoded image of the extracted chart in this case - this section will always be populated with a successful job.
- `content_metadata` - Describes the type of extraction and its position in the broader document - this section will always be populated with a successful job.
- `source_metadata` - Describes the source and storage path of an extracted chart in an S3 compliant object store.
- `table_metadata` - Contains the text representation of the chart, positional data, and other useful elements - this section will only exist when `metadata['content_metadata']['type'] == 'structured'`.

Note, `table_metadata` will store chart and table extractions. The are distringuished by `metadata['content_metadata']['subtype']`

In [ ]:
chart_metadata = generated_metadata[7]["metadata"]
chart_metadata_redact = chart_metadata.copy()
chart_metadata_redact["content"] = "<---Redacted for readability--->"
chart_metadata_redact

Visualize the chart contained within the extracted metadata.

In [ ]:
display.Image(b64decode(chart_metadata["content"]))

View the corresponding text that maps to this chart. This text could be embedded to support multimodal retrieval workflows.

In [ ]:
chart_metadata["table_metadata"]["table_content"]

### Explore Output - Images

This cell depicts the full metadata hierarchy for a image extraction with redacted content to ease readability. Notice the following sections are populated with information:

- `content` - The raw extracted content, a base64 encoded image extracted from the document in this case - this section will always be populated with a successful job.
- `content_metadata` - Describes the type of extraction and its position in the broader document - this section will always be populated with a successful job.
- `source_metadata` - Describes the source and storage path of an extracted image in an S3 compliant object store.
- `image_metadata` - Contains the image type, positional data, and other useful elements - this section will only exist when `metadata['content_metadata']['type'] == 'image'`.

In [ ]:
img_metadata = generated_metadata[1]["metadata"]
redact_metadata_helper(img_metadata)

Visualize the image contained within the extracted metadata.

In [ ]:
display.Image(b64decode(img_metadata["content"]))

### Optional:  Expanded Task Configuration

This section illustrates usage of the remaining task types used when supporting retrieval workflows.

- `StoreTask` - Stores extracted content to an S3 compliant object store (MinIO by default) and updates the `source_metadata` with the corresponding stored location.
- `EmbedTask` - Computes an embedding for the extracted content using a [`nvidia/nv-embedqa-e5-v5`](https://catalog.ngc.nvidia.com/orgs/nim/teams/nvidia/containers/nv-embedqa-e5-v5) NVIDIA Inference Microservice (NIM) by default.
- `VdbUploadTask` - Inserts ingested content into a Milvus vector database to support retrieval use cases.

Define the ingest job specification.

In [ ]:
job_spec = JobSpec(
    document_type=file_type,
    payload=file_content,
    source_id=SAMPLE_PDF,
    source_name=SAMPLE_PDF,
    extended_options={
        "tracing_options": {
            "trace": True,
            "ts_send": time.time_ns(),
        }
    },
)

Here the task configuration is expanded, but requires the ancillary services (Embedding NIM, MinIO object stor, and Milvus Vector Database) to be up and running to return metadata back to the client.

In [ ]:
extract_task = ExtractTask(
    document_type=file_type,
    extract_text=True,
    extract_images=True,
    extract_tables=True,
    text_depth="document",
    extract_tables_method="yolox",
)

dedup_task = DedupTask(
    content_type="image",
    filter=True,
)

filter_task = FilterTask(
    content_type="image",
    min_size=128,
    max_aspect_ratio=5.0,
    min_aspect_ratio=0.2,
    filter=True,
)

split_task = SplitTask(
    split_by="word",
    split_length=300,
    split_overlap=10,
    max_character_length=5000,
    sentence_window_size=0,
)

store_task = StoreTask(
    structured=True,
    images=True,
    store_method="minio",
    params={
        "access_key": MINIO_ACCESS_KEY, 
        "secret_key": MINIO_SECRET_KEY,
    }
)

embed_task = EmbedTask(
    text=True,
    tables=True,
)

store_embed_task = StoreEmbedTask(
    params={
        "access_key": MINIO_ACCESS_KEY, 
        "secret_key": MINIO_SECRET_KEY,
    }

)

vdb_upload_task = VdbUploadTask(
    bulk_ingest=True,
    bulk_ingest_path="embeddings/",
    params={
        "access_key": MINIO_ACCESS_KEY, 
        "secret_key": MINIO_SECRET_KEY,
    }
)


job_spec.add_task(extract_task)
job_spec.add_task(dedup_task)
job_spec.add_task(filter_task)
job_spec.add_task(split_task)
job_spec.add_task(store_task)
job_spec.add_task(embed_task)
job_spec.add_task(vdb_upload_task)

A job identifier is created for the job specification. This is used to retrieve the results upon completion.

With the `job_id`, the job is submitted to the NV-Ingest cluster. When the job is complete, the results are fetched.

In [ ]:
job_id = client.add_job(job_spec)
client.submit_job(job_id, TASK_QUEUE)
generated_metadata = client.fetch_job_result(
    job_id, timeout=DEFAULT_JOB_TIMEOUT
)[0][0]